In [1]:
import numpy as np
import math
import pandas as pd
import networkx as nx
import random
import collections
import matplotlib.pyplot as plt

# Trabajo final para evaluación de Métodos Numéricos de la Mestría de Ciencia de Datos (equipo 3)
* Jorge García (202945)
* Sandra España (203200)
* Aline Pérez (203096)
* Marco Antonio Ramos (142244)

Este trabajo busca implementar un problema de optimización de distancia recorrida en el metro de la CDMX.

# Creación de la matriz de adyacencias entre las estaciones del metro

Se obtuvo la base de datos de longitud entre estaciones del metro de la CDMX (long_interestacion) y la longitud de la estación (long_estacion). De la misma manera, se creó una lista con todas las estaciones del metro. Los datos originales se pueden consultar en la siguiente [liga](https://metro.cdmx.gob.mx/longitud-de-estacion)

In [2]:
metro1=pd.read_excel('Data1/distancia_metro.xlsx')
np.set_printoptions(precision=2)
nodos1=list(dict.fromkeys(pd.concat([metro1["Estacion1"],metro1["Estacion2"]])))
N=len(nodos1)

Se creó la matriz de adyacencias, en la cual los nodos son las estaciones y los valores son las distancias en metros

In [3]:
df1=np.zeros((N,N))
adyacencia1=pd.DataFrame(df1)
adyacencia1.columns=adyacencia1.index=nodos1

for i in nodos1:
    for j in nodos1:
        adyacencia1[i][j]=metro1[(metro1["Estacion1"]==i)&(metro1["Estacion2"]==j)]["Long_interestacion"]

adyacencia1=adyacencia1.fillna(0)
adyacencia1=np.transpose(adyacencia1)+adyacencia1
print(adyacencia1)

                      Pantitlan  Zaragoza  GomezFarias  BoulevardPuertoAereo  \
Pantitlan                   0.0    1320.0          0.0                   0.0   
Zaragoza                 1320.0       0.0        762.0                   0.0   
GomezFarias                 0.0     762.0          0.0                 611.0   
BoulevardPuertoAereo        0.0       0.0        611.0                   0.0   
Balbuena                    0.0       0.0          0.0                 595.0   
...                         ...       ...          ...                   ...   
MartinCarrera               0.0       0.0          0.0                   0.0   
BarrancadelMuerto           0.0       0.0          0.0                   0.0   
Constitucionde1917          0.0       0.0          0.0                   0.0   
LaPaz                       0.0       0.0          0.0                   0.0   
Buenavista                  0.0       0.0          0.0                   0.0   

                      Balbuena  Moctezu

Verificamos que no haya columnas que no tengan distancias en ninguna de sus entradas

In [4]:
print("máxima distancia por estación\n")
print("\n",np.max(adyacencia1))
print("\nverificamos que no haya un máximo igual a cero (es decir, que todas las estaciones tengan al menos una adyacencia)\n")
print("\n",np.sort(np.max(adyacencia1)))

máxima distancia por estación


 Pantitlan               1644.0
Zaragoza                1320.0
GomezFarias              762.0
BoulevardPuertoAereo     611.0
Balbuena                 703.0
                         ...  
MartinCarrera           1141.0
BarrancadelMuerto       1476.0
Constitucionde1917      1137.0
LaPaz                   1956.0
Buenavista               521.0
Length: 164, dtype: float64

verificamos que no haya un máximo igual a cero (es decir, que todas las estaciones tengan al menos una adyacencia)


 [ 445.  456.  516.  521.  564.  574.  587.  602.  611.  611.  611.  620.
  634.  637.  645.  653.  657.  657.  659.  665.  665.  698.  702.  703.
  703.  709.  709.  715.  725.  725.  732.  745.  745.  750.  755.  757.
  761.  762.  774.  788.  789.  793.  793.  794.  817.  817.  843.  860.
  884.  908.  908.  908.  910.  910.  924.  924.  942.  950.  959.  968.
  969.  969.  973.  993.  993. 1033. 1033. 1042. 1059. 1060. 1060. 1062.
 1067. 1072. 1084. 1106. 1110. 1110. 1111

# Creación del grafo de la matriz de adyacencias

In [5]:
def adj_matrix_2_edge_list(adj_matrix, index_name):
    """
    Esta función solamente transforma cualquier matriz cuadrada en una 
    lista de tuplas en el formato "edges list", lo cuál es muy útil
    para realizar los grafos. 
    
    Funciona iterando sobre los indices de la matriz, tomando i y j y 
    el valor respectivo. Elimina la traza para evitar grafos ciclicos.
    
    Index_name debe tener el mismo número de columnas que adj_matrix
    
    """
    edge_list = []
    matrix_len= len(adj_matrix)
    for i in range(matrix_len):
        for j in range(matrix_len):
                if i==j:
                    pass
                elif adj_matrix[i][j]==0:
                    pass
                else:
                    edge_list.append((index_name[i],index_name[j],adj_matrix[i][j]))
    edge_list = tuple(edge_list) 
    return edge_list

In [6]:
matrix = np.asarray(adyacencia1.values)

In [7]:
names = adyacencia1.index

In [8]:
names[1]

'Zaragoza'

In [9]:
G_d = adj_matrix_2_edge_list(matrix, names)

In [10]:
G_d[1:10]

(('Pantitlan', 'Hangares', 1644.0),
 ('Pantitlan', 'Puebla', 1380.0),
 ('Pantitlan', 'AgricolaOriental', 1409.0),
 ('Zaragoza', 'Pantitlan', 1320.0),
 ('Zaragoza', 'GomezFarias', 762.0),
 ('GomezFarias', 'Zaragoza', 762.0),
 ('GomezFarias', 'BoulevardPuertoAereo', 611.0),
 ('BoulevardPuertoAereo', 'GomezFarias', 611.0),
 ('BoulevardPuertoAereo', 'Balbuena', 595.0))

In [11]:
G = nx.Graph()

In [12]:
for node in G_d:
    G.add_node(node)

In [13]:
edges_list=adj_matrix_2_edge_list(matrix, names)
# tamaño de la matriz
#size=max([row[0] for row in edges_list])+1
size = len(names)

In [14]:
# Creamos el objeto grafo y le indicamos la cantidad de nodos
G = nx.Graph() 
node_list = range(size)
for node in node_list:
    G.add_node(node)

In [15]:
G.add_weighted_edges_from(edges_list)

In [16]:
nx.dijkstra_path(G, 'ElRosario', 'Deportivo18deMarzo')

['ElRosario',
 'Tezozomoc',
 'Azcapotzalco',
 'Ferreria',
 'Norte45',
 'Vallejo',
 'InstitutodelPetroleo',
 'Lindavista',
 'Deportivo18deMarzo']